<a href="https://colab.research.google.com/github/KValim/Projetcs/blob/main/Utilities/stock_prediction_with_LTSMtowardsdatascience.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://towardsdatascience.com/time-series-forecasting-with-recurrent-neural-networks-74674e289816

In [ ]:
# pip install jax==0.2.11

In [ ]:
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras import layers
import plotly.graph_objects as go

In [ ]:
filename = "comcast.csv"
df = pd.read_csv(filename)
df['Qtd Assinantes'] = df['Qtd Assinantes'].replace([38814,20000])
df = df[:-1]
print(df.info())

df['Date'] = pd.to_datetime(df['Date'])
df.set_axis(df['Date'], inplace=True)
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215 entries, 0 to 214
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Date            215 non-null    object
 1   Qtd Assinantes  215 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 3.5+ KB
None


,Date,Qtd Assinantes
Date,,
2004-10-01,2004-10-01,1629
2004-11-01,2004-11-01,3157
2004-12-01,2004-12-01,4186
2005-01-01,2005-01-01,4733
2005-02-01,2005-02-01,5247
...,...,...
2022-04-01,2022-04-01,9298
2022-05-01,2022-05-01,9125
2022-06-01,2022-06-01,8975


In [ ]:
close_data = df['Qtd Assinantes'].values
close_data = close_data.reshape((-1,1))

split_percent = 0.85
split = int(split_percent*len(close_data))

close_train = close_data[:split]
close_test = close_data[split:]

date_train = df['Date'][:split]
date_test = df['Date'][split:]

print(len(close_train))
print(len(close_test))

182
33


In [ ]:
look_back = 12

train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=20)     
test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=1)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(look_back,1)))
model.add(layers.Dense(25))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

num_epochs = 25
model.fit_generator(train_generator, epochs=num_epochs, verbose=1)

Epoch 1/25


<ipython-input-17-0a25bf74ec36>:11: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, epochs=num_epochs, verbose=1)


9/9 [==============================] - 2s 12ms/step - loss: 709904960.0000
Epoch 2/25
9/9 [==============================] - 0s 10ms/step - loss: 507101248.0000
Epoch 3/25
9/9 [==============================] - 0s 10ms/step - loss: 177616928.0000
Epoch 4/25
9/9 [==============================] - 0s 10ms/step - loss: 14557091.0000
Epoch 5/25
9/9 [==============================] - 0s 10ms/step - loss: 22974626.0000
Epoch 6/25
9/9 [==============================] - 0s 18ms/step - loss: 10974637.0000
Epoch 7/25
9/9 [==============================] - 0s 15ms/step - loss: 5340541.5000
Epoch 8/25
9/9 [==============================] - 0s 10ms/step - loss: 4563742.5000
Epoch 9/25
9/9 [==============================] - 0s 10ms/step - loss: 4488110.5000
Epoch 10/25
9/9 [==============================] - 0s 11ms/step - loss: 4443124.5000
Epoch 11/25
9/9 [==============================] - 0s 11ms/step - loss: 4501726.0000
Epoch 12/25
9/9 [==============================] - 0s 10ms/step - loss: 4361

In [ ]:
prediction = model.predict_generator(test_generator)

close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = prediction.reshape((-1))

trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "Google Stock",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()

<ipython-input-18-190da667821d>:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  prediction = model.predict_generator(test_generator)


In [ ]:
close_data = close_data.reshape((-1))

def predict(num_prediction, model):
    prediction_list = close_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = df['Date'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1, freq="M").tolist()
    return prediction_dates

num_prediction = 24
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)

1/1 [==============================] - 0s 20ms/step


In [ ]:
prediction = model.predict_generator(test_generator)

close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = prediction.reshape((-1))

trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = forecast_dates,
    y = forecast,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "Google Stock",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()

<ipython-input-20-25e6d3bf1ec4>:1: UserWarning:

`Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.

